#  **Efficient API Calls(Core)**

In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Imports/ Yelp
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
# Use secret credentials
with open('/Users/nicholasgiuffrida/.secret/yelp_api.json') as f:
    login = json.load(f)
# Create a Yelp API instance
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

##   Yelp API  search 
- favorite city for a cuisine type

In [3]:
# Test Query
results = yelp_api.search_query(location='Boston, MA',
                                       term='Chinese')
print(type(results))
results.keys()

<class 'dict'>


dict_keys(['businesses', 'total', 'region'])

In [4]:
# set our API call parameters
LOCATION = "Boston, MA"
TERM = "Chinese"

In [5]:
JSON_FILE = "Data/results_in_progress_Bos_chinese.json"
JSON_FILE

'Data/results_in_progress_Bos_chinese.json'

##  Extract results 

In [6]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    
    ## Check for JSON_FILE 
    file_exists = os.path.isfile(JSON_FILE)
    
    ## If it DOES exist:
    if file_exists == True:
        
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
            
            
    ## If it does NOT exist:
    else:
        
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [7]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Bos_chinese.json already exists. Deleting previous file...
[i] Data/results_in_progress_Bos_chinese.json not found. Saving empty list to new file.
- 0 previous results found.


32

In [8]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/32 [00:00<?, ?it/s]

##  Compile into  dataframe

In [9]:
# Load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,GCX6ycKIyIv4xNu_FuX1JA,jiangnan-boston-boston,JiangNan Boston,https://s3-media4.fl.yelpcdn.com/bphoto/Kj0iHQ...,False,https://www.yelp.com/biz/jiangnan-boston-bosto...,102,"[{'alias': 'chinese', 'title': 'Chinese'}]",4.5,"{'latitude': 42.353, 'longitude': -71.06406}",[],"{'address1': '177 Tremont St', 'address2': Non...",+18572770668,(857) 277-0668,823.451500,NaN
1,Xesye6qIiWLRxJ8XO8mo5g,dynasty-boston,Dynasty,https://s3-media3.fl.yelpcdn.com/bphoto/GzyJyY...,False,https://www.yelp.com/biz/dynasty-boston?adjust...,49,"[{'alias': 'cantonese', 'title': 'Cantonese'},...",4.5,"{'latitude': 42.350613, 'longitude': -71.0599079}",[],"{'address1': '14 Hudson St', 'address2': None,...",+16176038796,(617) 603-8796,979.179545,NaN
2,lJdytfj2BBFfv0_ax_P4WQ,dumpling-xuan-cambridge,Dumpling Xuan,https://s3-media4.fl.yelpcdn.com/bphoto/dtFsVT...,False,https://www.yelp.com/biz/dumpling-xuan-cambrid...,210,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 42.37165370284239, 'longitude': -...","[restaurant_reservation, delivery, pickup]","{'address1': '465 Cambridge St', 'address2': N...",+16178682888,(617) 868-2888,2848.031261,$$
3,oWxyhRtIQ2sk4Mbmsv1E0Q,hao-shi-guang-boston,Hao Shi Guang,https://s3-media3.fl.yelpcdn.com/bphoto/EM0XbP...,False,https://www.yelp.com/biz/hao-shi-guang-boston?...,21,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 42.35453, 'longitude': -71.13228}","[restaurant_reservation, delivery, pickup]","{'address1': '48 Harvard Ave', 'address2': '',...",+16179033327,(617) 903-3327,5071.342998,NaN
4,r-4e7Co5QyA0jfVRp3mRpw,zhi-wei-cafe-boston,Zhi Wei Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/bPoEEK...,False,https://www.yelp.com/biz/zhi-wei-cafe-boston?a...,21,"[{'alias': 'chinese', 'title': 'Chinese'}, {'a...",4.5,"{'latitude': 42.351081, 'longitude': -71.057132}",[],"{'address1': '104 South St', 'address2': None,...",+18572770937,(857) 277-0937,1208.740565,NaN


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
621,lTJ4yRH6bqNf1hF8Fnlz0Q,hong-kong-restaurant-newtonville,Hong Kong Restaurant,,False,https://www.yelp.com/biz/hong-kong-restaurant-...,1,"[{'alias': 'chinese', 'title': 'Chinese'}]",1.0,"{'latitude': 42.3504306, 'longitude': -71.2073...",[],"{'address1': '10 Austin St', 'address2': '', '...",+16175271100,(617) 527-1100,11198.973336,NaN
622,6Yynbq1Pr07gmJsQgI21Fw,bamboo-dragon-newton,Bamboo Dragon,,False,https://www.yelp.com/biz/bamboo-dragon-newton?...,4,"[{'alias': 'chinese', 'title': 'Chinese'}]",1.5,"{'latitude': 42.3494503, 'longitude': -71.2248...",[],"{'address1': '1251 Washington St', 'address2':...",+16179699019,(617) 969-9019,12629.462272,$
623,zjWnlUdlzXylXxcKcsUHKw,master-wok-saugus,Master Wok,https://s3-media1.fl.yelpcdn.com/bphoto/Bbu00X...,False,https://www.yelp.com/biz/master-wok-saugus?adj...,4,"[{'alias': 'chinese', 'title': 'Chinese'}]",1.0,"{'latitude': 42.4628967, 'longitude': -71.0249...",[delivery],"{'address1': '1277 Broadway', 'address2': '', ...",+17819412719,(781) 941-2719,13089.141190,NaN
624,Na6pAqIkbhuhIOBlWcrIOw,the-kebab-factory-somerville,The Kebab Factory,https://s3-media3.fl.yelpcdn.com/bphoto/cLTiB0...,False,https://www.yelp.com/biz/the-kebab-factory-som...,557,"[{'alias': 'indpak', 'title': 'Indian'}, {'ali...",3.5,"{'latitude': 42.3780965, 'longitude': -71.1065...","[pickup, delivery]","{'address1': '414 Washington St', 'address2': ...",+16173544996,(617) 354-4996,4436.601134,$$
625,SUz38qzIzB5vg9n-SPXuYw,rainbow-sushi-and-asian-cuisine-brookline-2,Rainbow Sushi & Asian Cuisine,https://s3-media1.fl.yelpcdn.com/bphoto/ELwgNI...,False,https://www.yelp.com/biz/rainbow-sushi-and-asi...,276,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 42.339272, 'longitude': -71.12087}","[pickup, delivery]","{'address1': '187 Harvard St', 'address2': '',...",+16172772999,(617) 277-2999,4191.323371,$$
